🚀 Monitoramento Automatizado de Preços: Mercado Livre & Cobrecom
🎯 Contexto e Objetivo
No setor de varejo e construção civil, a flutuação de preços e a concorrência são constantes. Este projeto automatiza o processo de pesquisa de mercado para cabos elétricos da marca Cobrecom, buscando garantir que a empresa ou o comprador tenha sempre o melhor custo-benefício em mãos.

O objetivo principal é ler uma lista de produtos desejados de um arquivo Excel, realizar buscas automatizadas no Mercado Livre, filtrar os resultados mais relevantes pelo menor preço e extrair especificações técnicas detalhadas para análise de métricas, como o valor por metro.

🛠️ Tecnologias Utilizadas
Python: Linguagem base para o desenvolvimento da lógica.

Pandas: Manipulação de DataFrames para entrada e saída de dados em Excel.

Selenium & WebDriver Manager: Automação da navegação web (Web Scraping), lidando com elementos dinâmicos e espera de carregamento.

Tratamento de Exceções (try-except): Garantia de que o código continue rodando mesmo se um produto específico não for encontrado.

⚙️ Funcionalidades do Projeto
Integração com Excel: O script lê uma base de dados (checar.xlsx) contendo o tipo de cabo, tamanho e cor.

Busca Inteligente: Realiza buscas automáticas combinando os dados do Excel com a marca específica.

Filtro Dinâmico de Menor Preço: O bot interage com a interface do Mercado Livre para ordenar os resultados do menor para o maior valor.

Validação de Correspondência: Verifica se o título do produto encontrado realmente contém o tamanho e a cor solicitados antes de prosseguir.

Extração de Especificações Técnicas: Captura dados profundos dentro da página do produto, como Tensão Nominal e Comprimento do Cabo.

Cálculo de Métrica de Valor: Calcula automaticamente o Valor por Metro, permitindo uma comparação justa entre cabos de comprimentos diferentes.

Relatório Final: Exporta todos os dados coletados (incluindo links e valores) para um novo arquivo Excel (cobrecom_ML.xlsx).

📊 Fluxo de Execução
Início: Inicialização do driver do Chrome e carregamento da base de dados.

Iteração: Para cada item na lista, o bot pesquisa no Mercado Livre.

Interação: Aceita cookies, ordena por preço e seleciona o item mais barato que atenda aos critérios.

Parsing: Coleta de preços e especificações técnicas dentro da página do anúncio.

Cálculo: Processamento matemático do custo unitário.

Conclusão: Fechamento do navegador e salvamento do arquivo final.

💡 Diferencial Técnico
Diferente de scrapers básicos, este projeto lida com:

Elementos Dinâmicos: Uso de WebDriverWait para garantir que o bot não quebre enquanto a página carrega.

Limpeza de Dados: Tratamento de strings e conversão de valores monetários (R$) para o formato float de cálculo.

Resiliência: Lógica para ignorar produtos não encontrados e seguir para o próximo item da lista sem interromper a execução total.

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

url = 'https://www.mercadolivre.com.br/'
marca = 'Cobrecom com proteção antichamas'

# Importando os arquivos
caminho = r'C:/Users/Base de Dados/'

produtos_df = pd.read_excel('checar.xlsx')
produtos_df = produtos_df
produtos_df = pd.DataFrame(produtos_df, columns=['CABO','TAMANHO','COR'])

#display(produtos_df)

for linha in produtos_df.index:
    driver.get(url)

    # Aceitar cookies
    try:
        botao_aceitar_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="action:understood-button"]')))
        botao_aceitar_cookies.click()  # Clicando no botão
        print("Cookies aceitos.")
    except Exception as e:
        print(f'Erro ao tentar aceitar cookies: {e}')
    
    # Pesquisar produto
    try:
        pesquisar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'nav-search-input')))
        pesquisa_completa = f"{produtos_df.loc[linha, 'CABO']} {produtos_df.loc[linha, 'TAMANHO']} {produtos_df.loc[linha, 'COR']} {marca}"
        pesquisar.send_keys(pesquisa_completa)
        driver.find_element(By.CLASS_NAME, 'nav-icon-search').click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-search-results')))  # Espera os resultados carregarem
        time.sleep(3)
    except Exception as e:
        print(f'Erro ao tentar pesquisar o produto: {e}')
        continue
    
    # Selecionando menor preço
    try:
        organizando = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'andes-dropdown__display-values')))
        organizando.click()
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id=":R2m55ee:-menu-list-option-price_asc"]/div/div/span').click()
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'poly-component__title')))  # Espera produtos aparecerem
        time.sleep(3)
    except Exception as e:
        print(f'Erro ao tentar selecionar menor preço: {e}')
        continue
    
    # Captura todos os produtos com a classe 'poly-component__title'
    try:
        produtos = driver.find_elements(By.CLASS_NAME, 'poly-component__title')
        encontrou_produto = False

        for produto in produtos:
            nome_produto = produto.text.lower()  # Convertendo para minúsculas para evitar problemas de capitalização
            tamanho = produtos_df.loc[linha, 'TAMANHO'].lower()
            cor = produtos_df.loc[linha, 'COR'].lower()

            # Verifica se o nome do produto contém o tamanho e cor desejados
            if tamanho in nome_produto and cor in nome_produto:
                # Se encontrar, clicar no produto
                produto.click()  # Clicando no produto
                print(f'Produto encontrado: {produto.get_attribute("href")}')
                encontrou_produto = True
                # Esperar a página do produto carregar completamente
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-pdp-title')))  # Espera o título do produto carregar
                break  # Se o produto for encontrado e clicado, sai do loop

        if not encontrou_produto:
            print(f'Nenhum produto correspondente encontrado para: {produtos_df.loc[linha, "CABO"]} {produtos_df.loc[linha, "TAMANHO"]} {produtos_df.loc[linha, "COR"]}')
    except Exception as e:
        print(f'Erro ao tentar capturar os produtos: {e}')
        continue

    # Pegar o valor do menor preço
    try:
        valor = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span[data-testid="price-part"]')))
        elemento_valor = valor.text.replace('\n', '').replace(' ', '').strip()
        menor_preco_valor = float(elemento_valor.replace('R$', '').replace('.', '').replace(',', '.').strip())
    except Exception as e:
        print(f'Erro ao tentar obter o valor do menor preço: {e}')
        continue

    # Pegar o nome do produto
    try:
        nome_produto = driver.find_element(By.CLASS_NAME, 'ui-pdp-header__title-container').text.strip()
    except Exception as e:
        print(f'Erro ao tentar obter o nome do produto: {e}')
        continue

    # Expor todas características do produto
    try:
        aumentar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="action-collapsable-target"]')))
        aumentar.click()
        time.sleep(2)
    except Exception as e:
        print(f'Erro ao tentar expor características do produto: {e}')
        continue

    # Pegar Tensão Nominal
    tensao_nominal = None
    try:
        linhas = driver.find_elements(By.CSS_SELECTOR, 'tbody.andes-table__body tr.ui-vpp-striped-specs__row')
        for linha in linhas:
            cabecalho = linha.find_element(By.TAG_NAME, 'th').text
            if cabecalho == 'Tensão nominal':
                tensao_nominal = linha.find_element(By.CSS_SELECTOR, 'td span').text.strip()
                break
    except Exception as e:
        print(f'Erro ao tentar obter a tensão nominal: {e}')

    # Pegar Comprimento do Cabo
    comprimento_cabo = None
    try:
        linhas = driver.find_elements(By.CSS_SELECTOR, 'tbody.andes-table__body tr.ui-vpp-striped-specs__row')
        for linha in linhas:
            cabecalho = linha.find_element(By.TAG_NAME, 'th').text
            if cabecalho == 'Comprimento do cabo':
                comprimento_cabo = linha.find_element(By.CSS_SELECTOR, 'td span').text.strip()
                break
    except Exception as e:
        print(f'Erro ao tentar obter o comprimento do cabo: {e}')

    # Agora, vamos tentar atribuir as variáveis ao DataFrame
    try:
        produtos_df.at[linha, 'Menor Preço'] = elemento_valor
        produtos_df.at[linha, 'Nome Loja'] = loja_venda
        produtos_df.at[linha, 'Número de Vendas'] = num_venda
        produtos_df.at[linha, 'Tensão Nominal'] = tensao_nominal
        produtos_df.at[linha, 'Comprimento do Cabo'] = comprimento_cabo
        produtos_df.at[linha, 'Link'] = menor_preco
        print(f"Informações adicionadas à linha {linha} do DataFrame.")
    except Exception as e:
        print(f"Erro ao adicionar informações ao DataFrame na linha {linha}: {e}")


    # Calcular o valor por metro e atribuir ao DataFrame
    if comprimento_valor:
        produtos_df.at[linha, 'Valor por Metro'] = menor_preco_valor / comprimento_valor
    else:
        produtos_df.at[linha, 'Valor por Metro'] = None  # Se comprimento não estiver disponível

# Verificar o DataFrame final
display(produtos_df)

# Fechar o driver
driver.quit()

In [ ]:
display(produtos_df)

In [ ]:
produtos_df.to_excel('cobrecom_ML.xlsx', index=False)